In [46]:
from qiskit import *
from qiskit.visualization import *
from qiskit.quantum_info import *
import numpy as np
import scipy
from scipy.optimize import minimize

class VariationalSwapTest:

    def __init__(self, num_target_qubits, num_layers,
                random_unitary_matrix, 
                simulator = BasicAer.get_backend('qasm_simulator')):

        self.num_target_qubits = num_target_qubits
        self.num_layers = num_layers
        self.random_unitary_matrix = random_unitary_matrix
        self.simulator = simulator

    def random_target_state(self):
        qreg  = QuantumRegister(self.num_target_qubits)
        qc = QuantumCircuit(qreg)
        qc.append(self.random_unitary_matrix, qreg)

        return qc
    
    def create_ansatz(self, params):

        qc = QuantumCircuit(self.num_target_qubits)

        if self.num_target_qubits == 1:
            # u3 = p(φ+π) sx p(ϴ+π) sx p(λ)
            qc.p(params[0]+np.pi, 0)
            qc.sx(0)
            qc.p(params[1]+np.pi,0)
            qc.sx(0)
            qc.p(params[2],0)


        else:
            for layer in range(self.num_layers):
                for param, qubit in enumerate(qc.qregs):
                    qc.rz(params[param + self.num_target_qubits * layer], qubit)
                    qc.ry(params[param + self.num_target_qubits * layer], qubit)
                    
                    
                    qc.u3(params[param + self.num_target_qubits * layer], 
                            params[1+ param + self.num_target_qubits * layer], 
                                params[2+ param + self.num_target_qubits * layer], 
                                    qubit)
                    

                if layer == self.num_layers - 1:
                    break

                for control in range(self.num_target_qubits):
                    for target in range(self.num_target_qubits):
                        if control < target:
                            qc.cx(control, target)

        return qc

    def variational_swap_test_circuit(self, parameters):

        meas_register = QuantumRegister(1)
        desired_register = QuantumRegister(self.num_target_qubits)
        ansatz_register = QuantumRegister(self.num_target_qubits)
        creg = ClassicalRegister(1)

        qc = QuantumCircuit(meas_register, desired_register, ansatz_register, creg)

        qc.append(self.random_target_state(), desired_register)
        qc.append(self.create_ansatz(parameters), ansatz_register)        
        qc.barrier()
        
        qc.h(meas_register[0])
        for i in range(self.num_target_qubits):
            qc.cswap(meas_register[0], desired_register[i], ansatz_register[i])
        qc.h(meas_register[0])

        qc.measure(meas_register[0], creg[0])

        return qc

    def cost_function(self, params):
        
        circuit = self.variational_swap_test_circuit(params)
        counts = execute(circuit, backend = self.simulator, shots = 10000).result().get_counts(circuit)

        if '0' not in counts:
            counts['0']=0
        if '1' not in counts:
            counts['1']=0

        total_counts = counts['0'] + counts['1']
        state_distance = 1 - (counts['0']/total_counts)
        print(f"Fidelity: \t {abs(1 - state_distance)}")

        return state_distance
    
    def minimize(self):

        convergence = []
        def callback(variational_parameters):
            convergence.append(self.cost_function(variational_parameters))
            print(f" \t Loss: {self.cost_function(variational_parameters)}")


        if self.num_target_qubits == 1:
            num_params = 3
        else:
            num_params = self.num_layers * self.num_target_qubits

        res = scipy.optimize.minimize(self.cost_function, x0=np.random.uniform(0, np.pi, num_params), 
                                        method = 'COBYLA', callback = callback,
                                        options={'maxiter': 30, 'ftol': 1e-08, 'iprint': 1, 'disp': True, 
                                        'eps': 1.4901161193847656e-08, 'finite_diff_rel_step': None})

        return res

    def show_quantum_states(self):

        result = self.minimize()
        target_circ = self.random_target_state()
        approx_circ = self.create_ansatz(result['x'])
        
        backend = Aer.get_backend('statevector_simulator')
        jobs = (execute(target_circ, backend=backend, shots=1, memory=True), 
                execute(approx_circ, backend=backend, shots=1, memory=True))
        
        job_result_target = jobs[0].result()
        job_result_approx = jobs[1].result()

        print(f"Target State: \t {job_result_target.get_statevector(target_circ)}")
        print(f"\nApprox. State: \t {job_result_approx.get_statevector(approx_circ)}")


In [50]:
random_U = random_unitary(2)
res =  VariationalSwapTest(1,1,random_U).minimize()

c:\Users\OS-5818\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:548: RuntimeWarning: Method COBYLA does not support callback.
  warn('Method %s does not support callback.' % method, RuntimeWarning)
c:\Users\OS-5818\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:629: OptimizeWarning: Unknown solver options: ftol, iprint, eps, finite_diff_rel_step
  return _minimize_cobyla(fun, x0, args, constraints, **options)


Fidelity: 	 0.707
Fidelity: 	 0.7037
Fidelity: 	 0.5645
Fidelity: 	 0.7118
Fidelity: 	 0.9288
Fidelity: 	 0.9817
Fidelity: 	 0.8506
Fidelity: 	 0.9821
Fidelity: 	 0.9177
Fidelity: 	 0.9872
Fidelity: 	 0.968
Fidelity: 	 0.9835
Fidelity: 	 0.9644
Fidelity: 	 0.9877
Fidelity: 	 0.9949
Fidelity: 	 0.9991
Fidelity: 	 0.9848
Fidelity: 	 0.9994
Fidelity: 	 0.9994
Fidelity: 	 0.9957
Fidelity: 	 0.9994
Fidelity: 	 0.999
Fidelity: 	 0.9995
Fidelity: 	 0.9994
Fidelity: 	 0.9997
Fidelity: 	 0.9998
Fidelity: 	 0.9996
Fidelity: 	 0.9995
Fidelity: 	 0.9996
Fidelity: 	 0.9993
Fidelity: 	 0.9996
Fidelity: 	 0.9996
Fidelity: 	 0.9997
Fidelity: 	 0.9996
Fidelity: 	 0.9998
Fidelity: 	 0.9997
Fidelity: 	 0.9996
Fidelity: 	 0.9991
Fidelity: 	 0.9994
Fidelity: 	 0.9996
Fidelity: 	 0.9997
Fidelity: 	 0.9998
Fidelity: 	 0.9997
Fidelity: 	 0.9996
Fidelity: 	 0.9995
Fidelity: 	 0.9994
Fidelity: 	 0.9996


In [52]:
from itertools import count
circ = VariationalSwapTest(1,1, random_U).variational_swap_test_circuit(res['x'])
circ.draw()


░ ┌───┐   ┌───┐┌─┐
q27372: ──────────────────░─┤ H ├─■─┤ H ├┤M├
        ┌───────────────┐ ░ └───┘ │ └───┘└╥┘
q27373: ┤ circuit-77921 ├─░───────X───────╫─
        ├───────────────┤ ░       │       ║ 
q27374: ┤ circuit-77924 ├─░───────X───────╫─
        └───────────────┘ ░               ║ 
c871: 1/══════════════════════════════════╩═
                                          0

In [53]:
counts = execute(circ, backend = BasicAer.get_backend('qasm_simulator'), shots= 1000).result().get_counts(circ)
print(counts)

{'0': 1000}
